In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from urllib.request import urlopen
from bs4 import BeautifulSoup
import pandas as pd

//anaconda3/lib/python3.7/site-packages/joblib/_multiprocessing_helpers.py:53: UserWarning: [Errno 28] No space left on device.  joblib will operate in serial mode
  warnings.warn('%s.  joblib will operate in serial mode' % (e,))


In [3]:
# NBA season we will be analyzing
year = 2019
# URL page we will scraping (see image above)
url = "https://www.basketball-reference.com/leagues/NBA_{}_per_game.html".format(year)
# this is the HTML from the given URL
html = urlopen(url)
soup = BeautifulSoup(html)

# use findALL() to get the column headers
soup.findAll('tr', limit=2)
# use getText()to extract the text we need into a list
headers = [th.getText() for th in soup.findAll('tr', limit=2)[0].findAll('th')]
# exclude the first column as we will not need the ranking order from Basketball Reference for the analysis
headers = headers[1:]
headers

# avoid the first header row
rows = soup.findAll('tr')[1:]
player_stats = [[td.getText() for td in rows[i].findAll('td')]
            for i in range(len(rows))]
stats = pd.DataFrame(player_stats, columns = headers)
stats.set_index("Player")



,Pos,Age,Tm,G,GS,MP,FG,FGA,FG%,3P,...,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS
Player,,,,,,,,,,,,,,,,,,,,,
Álex Abrines,SG,25,OKC,31,2,19.0,1.8,5.1,.357,1.3,...,.923,0.2,1.4,1.5,0.6,0.5,0.2,0.5,1.7,5.3
Quincy Acy,PF,28,PHO,10,0,12.3,0.4,1.8,.222,0.2,...,.700,0.3,2.2,2.5,0.8,0.1,0.4,0.4,2.4,1.7
Jaylen Adams,PG,22,ATL,34,1,12.6,1.1,3.2,.345,0.7,...,.778,0.3,1.4,1.8,1.9,0.4,0.1,0.8,1.3,3.2
Steven Adams,C,25,OKC,80,80,33.4,6.0,10.1,.595,0.0,...,.500,4.9,4.6,9.5,1.6,1.5,1.0,1.7,2.6,13.9
Bam Adebayo,C,21,MIA,82,28,23.3,3.4,5.9,.576,0.0,...,.735,2.0,5.3,7.3,2.2,0.9,0.8,1.5,2.5,8.9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Tyler Zeller,C,29,MEM,4,1,20.5,4.0,7.0,.571,0.0,...,.778,2.3,2.3,4.5,0.8,0.3,0.8,1.0,4.0,11.5
Ante Žižić,C,22,CLE,59,25,18.3,3.1,5.6,.553,0.0,...,.705,1.8,3.6,5.4,0.9,0.2,0.4,1.0,1.9,7.8
Ivica Zubac,C,21,TOT,59,37,17.6,3.6,6.4,.559,0.0,...,.802,1.9,4.2,6.1,1.1,0.2,0.9,1.2,2.3,8.9


In [17]:
stats.lookup('James Harden')

TypeError: lookup() missing 1 required positional argument: 'col_labels'

In [5]:
year = 2018
# URL page we will scraping (see image above)
url = "https://www.basketball-reference.com/leagues/NBA_{}_per_game.html".format(year)
# this is the HTML from the given URL
html = urlopen(url)
soup = BeautifulSoup(html)

# use findALL() to get the column headers
soup.findAll('tr', limit=2)
# use getText()to extract the text we need into a list
headers = [th.getText() for th in soup.findAll('tr', limit=2)[0].findAll('th')]
# exclude the first column as we will not need the ranking order from Basketball Reference for the analysis
headers = headers[1:]
headers

# avoid the first header row
rows = soup.findAll('tr')[1:]
player_stats = [[td.getText() for td in rows[i].findAll('td')]
            for i in range(len(rows))]
stats2 = pd.DataFrame(player_stats, columns = headers)
stats2.set_index("Player")
combined = pd.concat([stats,stats2], axis=0)

In [8]:
combined.loc['James Harden']

KeyError: 'James Harden'